# Co-factor swap

Some text

In [24]:
# Load libraries
import numpy as np
from cobra.io import read_sbml_model, write_sbml_model
from cobra.util import create_stoichiometric_matrix
from cobra import Reaction, Metabolite
from cobra.core.gene import GPR
from cameo.strain_design.heuristic.evolutionary_based import CofactorSwapOptimization
from cameo.strain_design.heuristic.evolutionary.optimization import CofactorSwapOptimization
from cameo.strain_design.heuristic.evolutionary.objective_functions import product_yield, biomass_product_coupled_yield
from cameo.util import TimeMachine
from cameo.flux_analysis.analysis import flux_variability_analysis as fva
from cameo import load_model

In [25]:
# Loading model
infilename = 'models/iMM904_progesterone.xml'
print(f"Loading {infilename}")
model = load_model(infilename)

Loading models/iMM904_progesterone.xml


In [26]:
# Add demand reaction for progesterone
model.add_boundary(model.metabolites.get_by_id('progesterone_c'), type='demand')
print(f'Adding reaction {model.reactions.DM_progesterone_c.name}: {model.reactions.DM_progesterone_c.reaction}')

Adding reaction progesterone demand: progesterone_c --> 


In [27]:
# Finding the optimal fluxes through each reaction when the objective is biomass production 
Fluxes = model.optimize()
Fluxes

,fluxes,reduced_costs
CITtcp,0.00000,0.000000
13BGH,0.00000,-0.047516
13BGHe,0.00000,0.000000
13GS,0.32667,0.000000
16GS,0.00000,0.000000
...,...,...
R03353,0.00000,0.000000
R07215,0.00000,0.000000
R04804,0.00000,0.000000
SK_methylpentanal_c,0.00000,0.000000


In [28]:
# Choose the reaction that produced progesterone as the objective.
objective_reaction = model.reactions.R02216
model.objective = objective_reaction
print(f'Objective set to maximize {objective_reaction.id}: {objective_reaction.reaction}')

Objective set to maximize R02216: nad_c + pregnenolone_c <=> h_c + nadh_c + progesterone_c


In [29]:
# Setting the progesterone yield from glucose as substrate
py = product_yield(model.reactions.R02216, model.exchanges.EX_glc__D_e)
py

In [59]:
#optswap

In [30]:
# Find reactions that all have the targeted co-factor pairs and add reactions that have the co-factors swapped.
swap_1 = CofactorSwapOptimization(model=model, cofactor_id_swaps=(["nad_c","nadh_c"],["nadp_c","nadph_c"]),objective_function=py)

swap_1

In [31]:
# Identify the reactions that when swapped to another co-factor improve the production of progesterone
solution = swap_1.run(max_size = 20)

Starting optimization at Fri, 18 Nov 2022 13:22:57


HBox()

Finished after 00:03:35


In [32]:
solution

,index,targets,fitness
0,0,"(MTHFD, GAPD)",0.358536
1,1,"(GAPD, ASAD)",0.358536
2,2,"(GAPD, MTHFD)",0.350835
3,3,"(GAPD, GLUDy)",0.343458
4,4,"(GAPD,)",0.342857
5,7,"(ALCD2x_copy1,)",0.323077
6,9,"(ALCD2x_copy2,)",0.323077
7,12,"(MTHFD, MTHFD2i)",0.323077
8,20,"(HMGCOAR, C3STDH2, LNS14DM, C4STMO1, R07498)",0.278145
9,21,"(C3STDH1, HMGCOAR)",0.257668


In [33]:
solution.model

Name,iMM904_progesterone
Memory address,0x07feec9556fa0
Number of metabolites,1234
Number of reactions,1587
Number of groups,0
Objective expression,1.0*R02216 - 1.0*R02216_reverse_2ed70
Compartments,"cytosol, extracellular space, mitochondria, peroxisome/glyoxysome, endoplasmic reticulum, vacuole, golgi apparatus, nucleus"


In [88]:
solution.model

Name,iMM904
Memory address,0x07f131e622f40
Number of metabolites,1235
Number of reactions,1588
Number of groups,0
Objective expression,1.0*R02216 - 1.0*R02216_reverse_2ed70
Compartments,"cytosol, extracellular space, mitochondria, peroxisome/glyoxysome, endoplasmic reticulum, vacuole, golgi apparatus, nucleus"


In [35]:
reactions=solution.data_frame.targets
reactions

0                                    (MTHFD, GAPD)
1                                     (GAPD, ASAD)
2                                    (GAPD, MTHFD)
3                                    (GAPD, GLUDy)
4                                          (GAPD,)
5                                  (ALCD2x_copy1,)
6                                  (ALCD2x_copy2,)
7                                 (MTHFD, MTHFD2i)
8     (HMGCOAR, C3STDH2, LNS14DM, C4STMO1, R07498)
9                               (C3STDH1, HMGCOAR)
10                                      (HMGCOAR,)
11                             (ECYP11A1, C3STKR2)
12                                     (ECYP11A1,)
13              (C14STR, R05703, ECYP11A1, R07215)
14                       (C4STMO2, R01456, R02216)
15                         (C4STMO1, R05703, SQLS)
16                                      (C4STMO2,)
17                      (C3STDH1, C3STKR1, R07215)
18                               (C3STDH1, R02216)
19                             

In [36]:
for t in reactions:
    print("Reactions:")
    for id in t:
        r=model.reactions.get_by_id(id)
        #print(r.name, id+",","Fitness:", solution.data_frame.fitness[id][10])
        s=""
        for rea in range(len(r.reactants)):
            if rea==len(r.reactants)-1:
                s+=r.reactants[rea].name+" ==> "
            else:
                s+=r.reactants[rea].name+" + "
        for p in range(len(r.products)):
            if p==len(r.products)-1:
                s+=r.products[p].name
            else:
                s+=r.products[p].name+" + "
        print(s)
        print()

Reactions:
5,10-Methylenetetrahydrofolate + Nicotinamide adenine dinucleotide phosphate ==> 5,10-Methenyltetrahydrofolate + Nicotinamide adenine dinucleotide phosphate - reduced

Glyceraldehyde 3-phosphate + Phosphate + Nicotinamide adenine dinucleotide ==> 3-Phospho-D-glyceroyl phosphate + H+ + Nicotinamide adenine dinucleotide - reduced

Reactions:
Glyceraldehyde 3-phosphate + Phosphate + Nicotinamide adenine dinucleotide ==> 3-Phospho-D-glyceroyl phosphate + H+ + Nicotinamide adenine dinucleotide - reduced

L-Aspartate 4-semialdehyde + Phosphate + Nicotinamide adenine dinucleotide phosphate ==> 4-Phospho-L-aspartate + H+ + Nicotinamide adenine dinucleotide phosphate - reduced

Reactions:
Glyceraldehyde 3-phosphate + Phosphate + Nicotinamide adenine dinucleotide ==> 3-Phospho-D-glyceroyl phosphate + H+ + Nicotinamide adenine dinucleotide - reduced

5,10-Methylenetetrahydrofolate + Nicotinamide adenine dinucleotide phosphate ==> 5,10-Methenyltetrahydrofolate + Nicotinamide adenine din